In [1]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import pandas as pd
import numpy as np

# takes FASTA files and generates dataframe and csv with descriptions and features

In [ ]:
# prot_features = {
#     "Molecular Weight": lambda x: ProteinAnalysis(str(x)).molecular_weight(),
#     "Aromaticity": lambda x: ProteinAnalysis(str(x)).aromaticity(),
#     "Instability Index": lambda x: ProteinAnalysis(str(x)).instability_index(),
#     "Flexibility": lambda x: ProteinAnalysis(str(x)).flexibility(),
#     "GRAVY": lambda x: ProteinAnalysis(str(x)).gravy(),
#     "Isoelectric Point": lambda x: ProteinAnalysis(str(x)).isoelectric_point(),
#     "Charge at pH:7.0": lambda x: ProteinAnalysis(str(x)).charge_at_pH(7.0),
# }
prot_feats = {
    "Molecular Weight": lambda x: x.molecular_weight(),
    "Aromaticity": lambda x: x.aromaticity(),
    "Instability Index": lambda x: x.instability_index(),
    "Flexibility": lambda x: x.flexibility(),
    "GRAVY": lambda x: x.gravy(),
    "Isoelectric Point": lambda x: x.isoelectric_point(),
    "Charge at pH:7.0": lambda x: x.charge_at_pH(7.0),
}

In [ ]:
input_file = "filtered_len_less_500.fasta"

records = []
lim = 10000
for record in SeqIO.parse(input_file, "fasta"):
    seq_str = str(record.seq)

    row = {
        "Sequence": seq_str,
        "ID": record.id,
        "Name": record.name,
        "Description": record.description,
        "Length": len(seq_str),
        "Num_Features": len(getattr(record, "features", [])),  # robust to missing 'features'
    }

    # Compute protein features unless there's an 'X' (unknown amino acid)
    prot = ProteinAnalysis(seq_str)
    if "X" in seq_str or "U" in seq_str:
        for feature_name in prot_feats:
            row[feature_name] = np.nan
    else:
        for feature_name, feature_func in prot_feats.items():
            row[feature_name] = feature_func(prot)

    records.append(row)
    lim -= 1
    if lim == 0:
        break

# Create DataFrame in one go (much faster)
seq_df = pd.DataFrame(records)
seq_df.set_index("ID", inplace=True)
seq_df.head()

In [ ]:
seq_df.to_csv("seq_df_first_20000_less_500.csv", index=True, compression=None)

In [ ]:
first_5000 = seq_df.head(2000)
first_5000.to_csv("seq_df_first_2000.csv", index=True)

In [ ]:
seq_df